## Import Libraries

In [105]:
import os
from crewai import Agent, Task, Crew
from crewai.flow.flow import Flow, start, listen  
from langchain_experimental.tools import PythonREPLTool
from langchain_community.tools import DuckDuckGoSearchRun



## Load the  LLM


In [106]:
from crewai import LLM

llm = LLM(
    model="groq/qwen/qwen3-32b",               # Updated model name
    api_key=os.getenv("GROQ_API_KEY"),
    api_base="https://api.groq.com/openai/v1"
)


## Load Tools

In [ ]:
from crewai.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun

@tool("DuckDuckGo Search")
def search_tool1(query: str) -> str:
    """Searches the web using DuckDuckGo."""
    time.sleep(1.0 + random.random()*0.5) 
    return DuckDuckGoSearchRun().run(query)



## Agent Definitions



In [108]:
loader_agent = Agent(
    role="Loader Agent",
    goal="Search and retrieve electric consumption data for Indian states (2018-2023).",
    backstory="A research assistant who finds credible data sources from the web.",
    tools=[search_tool1],
    llm=llm,
    verbose=True
)

In [109]:
from langchain_experimental.tools import PythonREPLTool


@tool("analyzer")
def search_tool2(query: str) -> str:
    """Executes Python code for data analysis or visualization."""
    return PythonREPLTool().run(query)


In [110]:
analyzer_agent = Agent(
    role="Analyzer Agent",
    goal="Analyze the CSV to find top 3 states with highest average consumption.",
    backstory="A data analyst who uses pandas to extract patterns.",
    tools=[search_tool2],
    llm=llm,
    verbose=True
)
#This one loads CSV and analyzes with pandas.

In [111]:
visualizer_agent = Agent(
    role="Visualizer Agent",
    goal="Create bar charts to show electricity consumption per year by state.",
    backstory="A matplotlib expert for Indian state electricity data.",
    tools=[search_tool2],
    llm=llm,
    verbose=True
)
#Makes visual plots of electricity data.

In [112]:
summarizer_agent = Agent(
    role="Summarizer Agent",
    goal="Write a 2-3 sentence summary about which states consumed the most electricity and why.",
    backstory="A professional executive summary writer.",
    tools=[],
    llm=llm,
    verbose=True
)
#Pure LLM-based summary writer.



## Task Definitions

In [113]:
load_task = Task(
    description="Search for electricity consumption datasets for Indian states (2018–2023).",
    agent=loader_agent,
    expected_output="A list of 3 URLs to government or trusted sources",
    input_type="input"
    )


analyze_task = Task(
    description="Load a CSV file, calculate average consumption, and print top 3 states.",
    agent=analyzer_agent,
    expected_output="A list of the top 3 Indian states with the highest average electricity consumption."
)

visualize_task = Task(
    description="Use matplotlib/seaborn to plot electricity consumption for each state.",
    agent=visualizer_agent,
    expected_output="A bar chart or line graph showing electricity consumption for each Indian state."
)

summarize_task = Task(
    description="Write a summary of insights from the data and visualizations in 2-3 sentences.",
    agent=summarizer_agent,
    expected_output="A concise textual summary of the most important insights from the analysis and visualizations."
)


combine_task = Task(
    description="Combine outputs from analyzer, visualizer, and summarizer into a single report.",
    agent=summarizer_agent,
    # optional: context=[analyze_task, visualize_task, summarize_task],  # not required with sequential
    expected_output="A single, well-structured final report summarizing the data, chart insights, and top 3 states."
)



## Flow Definition

In [114]:
from crewai import Crew, Process

#  Step 1: Run the loader agent first (sequential)
loader_crew = Crew(
    agents=[loader_agent],
    tasks=[load_task],
    process=Process.sequential
)

#  Get the context (output from loader_agent)
context = loader_crew.kickoff(inputs={"input": "Search for electricity consumption datasets for Indian states (2018–2023). Focus on government or trusted sources. Return top 3 links."})



#  Step 2: Set the next tasks to run in parallel
analyze_task.async_execution = True
visualize_task.async_execution = True
summarize_task.async_execution = True
combine_task.async_execution = False

#  Step 3: Run the 3 agents in parallel using shared context
main_crew = Crew(
    agents=[analyzer_agent, visualizer_agent, summarizer_agent],
    tasks=[analyze_task, visualize_task, summarize_task,combine_task],
    process=Process.sequential
)


# Already ran loader_crew above
context = loader_crew.kickoff(inputs={
    "input": "Search for electricity consumption datasets for Indian states (2018–2023). Focus on government or trusted sources. Return top 3 links."
})

# Convert CrewOutput to plain string
text = context.raw

# Now run main_crew with string input
final_result = main_crew.kickoff(inputs={"input": text})
print(final_result)




╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Loader Agent                                                                                            │
│                                                                                                                 │
│  Task: Search for electricity consumption datasets for Indian states (2018–2023).                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

 

I encountered an error while trying to use the tool. This was the error: https://html.duckduckgo.com/html 202 Ratelimit.
 Tool DuckDuckGo Search accepts these inputs: Tool Name: DuckDuckGo Search
Tool Arguments: {'query': {'description': None, 'type': 'str'}}
Tool Description: Searches the web using DuckDuckGo.



╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Loader Agent                                                                                            │
│                                                                                                                 │
│  Thought: <think>                                                                                               │
│  Okay, let's see. I need to find electricity consumption data for Indian states from 2018 to 2023. The user     │
│  wants three URLs from government or trusted sources. First, I should figure out where to search. The main      │
│  government sources in India for such data might be the Ministry of Power, maybe the Central Electricity        │
│  Authority (CEA), or state electricity regulatory commissions.                                                  │
│  I remember that the Ministry of Power's website has reports and statistics. Also, the Central Electricity      │
│  Authority publishes load despatch data. Another possibility is the Power Statistics reports they release       │
│  yearly. Also, the Indian Electricity Association might have data, but I'm not sure if that's considered        │
│  government. Maybe the Central Electricity Regulatory Commission (CERC) or state-level entities like SERCs.     │
│  Another thought is the National Statistical Office (NSO) or the Economic Survey of India, but those might not  │
│  have state-specific granular data.                                                                             │
│  Let me start by searching for official sources. A DuckDuckGo search for "Indian Ministry of Power electricity  │
│  consumption data 2018-2023" might lead to their website. Alternatively, searching for "Central Electricity     │
│  Authority India annual reports 2018-2023". Also, maybe the "Power Statistics 2023" report from the Ministry.   │
│  Also, state energy departments might publish their own data, but getting data for all states would be          │
│  tedious.                                                                                                       │
│  Let me try a search term: "India state electricity consumption data 2018 to 2023 government sources". Let me   │
│  see what comes up. The CEA's portal might have load data. For example, the CEA's "Power System Performance     │
│  Indicators". Also, the Power System Operation Corporation Limited (POSOCO) might have some datasets. Another   │
│  possibility is the "Annual Report of Ministry of Power India" which might include data. Also, the National     │
│  Electric Database or Energy Statistics from the Department of Economic Affairs.                                │
│  Wait, the user specified government or trusted sources. So the Ministry of Power, CEA, CERC, etc. Let me try   │
│  the first search. Then see if I can find three reliable URLs. Maybe the Ministry of Power's website has a      │
│  data portal. Let me check. For example, the Ministry's "Statistics and Reports" section. Also, the Central     │
│  Electric Authority's reports. Also, the Power Data Portal by the Government of India. Maybe the "Energy        │
│  Statistics" publication by the NSO. Alternatively, the "Electricity Consumption State Wise 2018-2023" on the   │
│  Ministry's website. Let me search for exact URLs now.                                                          │
│  </think>                                                                                                       │
│  Thought: I will search for recent and official sources related to Indian state electricity consumption data    │
│  between 2018–2023.                                                                                             │
│                                                        

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"query\": \"India state electricity consumption data 2018 to 2023 government sources\"}"                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│  I encountered an error while trying to use the tool. This was the error: https://html.duckduckgo.com/html 202  │
│  Ratelimit.                                                                                                     │
│   Tool DuckDuckGo Search accepts these inputs: Tool Name: DuckDuckGo Search                                     │
│  Tool Arguments: {'query': {'description': None, 'type': 'str'}}                                                │
│  Tool Description: Searches the web using DuckDuckGo..                                                          │
│  Moving on then. I MUST either use a tool (use one at time) OR give my best final answer not both at the same   │
│  time. When responding, I must use the following format:                                                        │
│                                                                                                                 │
│  ```                                                                                                            │
│  Thought: you should always think about what to do                                                              │
│  Action: the action to take, should be one of [DuckDuckGo Search]                                               │
│  Action Input: the input to the action, dictionary enclosed in curly braces                                     │
│  Observation: the result of the action                                                                          │
│  ```                                                                                                            │
│  This Thought/Action/Action Input/Result can repeat N times. Once I know the final answer, I must return the    │
│  following format:                                                                                              │
│                                                                                                                 │
│  ```                                                                                                            │
│  Thought: I now can give a great answer                                                                         │
│  Final Answer: Your final answer must be the great and the most complete as possible, it must be outcome        │
│  described                                                                                                      │
│                                                                                                                 │
│  ```                                                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Analyzer Agent                                                                                          │
│                                                                                                                 │
│  Thought: <think>                                                                                               │
│  Okay, I need to figure out why there's an ImportError with numpy.core.multiarray. The user tried to run a      │
│  pandas operation, which usually depends on numpy. The error suggests that numpy isn't properly installed or    │
│  there's a version conflict.                                                                                    │
│  First, maybe the environment doesn't have numpy installed. Since pandas requires numpy, that's a problem. The  │
│  user should install numpy using pip. Let me check if that's the case. If the error is about numpy not being    │
│  present, installing numpy should resolve it.                                                                   │
│  Alternatively, there might be an issue with the pandas or numpy installation. Perhaps a corrupted install.     │
│  Reinstalling pandas might help. But the most straightforward solution is to install numpy first.               │
│  Wait, maybe the user didn't install numpy at all. So adding pip install numpy before running the code could    │
│  fix the error. Let me proceed with that. Once numpy is installed, rerunning the pandas code should work.       │
│  Then, the code should group by State, calculate the mean of Consumption, sort descending, take top 3, and      │
│  return the state names.                                                                                        │
│  </think>                                                                                                       │
│  Thought: The error is due to missing NumPy dependency. Need to install NumPy first since Pandas depends on     │
│  it. Let me install NumPy and try again.                                                                        │
│                                                                                                                 │
│  Using Tool: analyzer                                                                                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"query\": \"import subprocess; subprocess.run(['pip', 'install', 'numpy'], check=True)\"}"                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Exception in thread Thread-12 (_execute_task_async):
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/site-packages/litellm/llms/custom_httpx/llm_http_handler.py", line 171, in _make_common_sync_call
    response = sync_httpx_client.post(
               ^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/site-packages/litellm/llms/custom_httpx/http_handler.py", line 780, in post
    raise e
  File "/opt/anaconda3/lib/python3.12/site-packages/litellm/llms/custom_httpx/http_handler.py", line 762, in post
    response.raise_for_status()
  File "/opt/anaconda3/lib/python3.12/site-packages/httpx/_models.py", line 761, in raise_for_status
    raise HTTPStatusError(message, request=request, response=self)
httpx.HTTPStatusError: Client error '429 Too Many Requests' for url 'https://api.groq.com/openai/v1/chat/completions'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/429

During handling of the above exception, another ex

KeyboardInterrupt: 

##  Run and Show Result

In [ ]:
crew = Crew(flow=flow, verbose=True)
result = crew.kickoff()

print("\n Final Result:ssssss


SyntaxError: unterminated string literal (detected at line 4) (2811737534.py, line 4)

HTTPSConnectionPool(host='telemetry.crewai.com', port=4319): Read timed out. (read timeout=29.999995946884155)
